# Create Networks - Advanced

This tutorial shows how to create a more complex pandapower network step by step. The network includes every element which is availiable in the pandapower framework.

The final network looks like this:

<img src="pics/example_network.png" width="50%">

The structural information about this network are stored in csv tables in the example_advanced folder.

For a better overview the creation of the individual components is divided in three steps. Each step handles one of the three voltage levels: high, medium and low voltage. We star by initializing an empty pandapower network:

In [52]:
#import the pandapower module
import pandapower as pp
import pandas as pd

#create an empty network 
net = pp.create_empty_network()

## High voltage level

### Buses

<img src="pics/example_network_buses_hv_detail.png" width="50%">


There are two 380 kV and five 110 kV busbars (type="b"). The 380/110 kV substation is modeled in detail with all nodes and switches, which is why we need additional nodes (type="b") to connect the switches.

In [53]:
# Double busbar
pp.create_bus(net, name='Double Busbar 1', vn_kv=380, type='b')
pp.create_bus(net, name='Double Busbar 2', vn_kv=380, type='b')
for i in range(10):
    pp.create_bus(net, name='Bus DB T%s' % i, vn_kv=380, type='n')
for i in range(1, 5):
    pp.create_bus(net, name='Bus DB %s' % i, vn_kv=380, type='n')

# Single busbar
pp.create_bus(net, name='Single Busbar', vn_kv=110, type='b')
for i in range(1, 6):
    pp.create_bus(net, name='Bus SB %s' % i, vn_kv=110, type='n')
for i in range(1, 6):
    for j in [1, 2]:
        pp.create_bus(net, name='Bus SB T%s.%s' % (i, j), vn_kv=110, type='n')

# Remaining buses
for i in range(1, 5):
    pp.create_bus(net, name='Bus HV%s' % i, vn_kv=110, type='n')

# show bustable
net.bus

,name,vn_kv,type,zone,in_service
0,Double Busbar 1,380.0,b,None,True
1,Double Busbar 2,380.0,b,None,True
2,Bus DB T0,380.0,n,None,True
3,Bus DB T1,380.0,n,None,True
4,Bus DB T2,380.0,n,None,True
5,Bus DB T3,380.0,n,None,True
6,Bus DB T4,380.0,n,None,True
7,Bus DB T5,380.0,n,None,True
8,Bus DB T6,380.0,n,None,True
9,Bus DB T7,380.0,n,None,True


### Lines

<img src="pics/example_network_lines_hv.png" width="40%">

The information about the 6 HV lines are stored in a csv file that we load from the hard drive:

In [54]:
hv_lines = pd.read_csv('example_advanced/hv_lines.csv', sep=';', header=0, decimal=',')
hv_lines

,line_name,from_bus,to_bus,std_type,length,parallel
0,HV Line1,Bus SB 2,Bus HV1,184-AL1/30-ST1A 110.0,30,1
1,HV Line2,Bus HV1,Bus HV2,184-AL1/30-ST1A 110.0,20,1
2,HV Line3,Bus HV2,Bus HV4,184-AL1/30-ST1A 110.0,30,1
3,HV Line4,Bus HV1,Bus HV4,184-AL1/30-ST1A 110.0,15,1
4,HV Line5,Bus HV3,Bus HV4,184-AL1/30-ST1A 110.0,25,1
5,HV Line6,Bus SB 3,Bus HV3,184-AL1/30-ST1A 110.0,30,2


and use to create all lines:

In [55]:
# create lines
for _, hv_line in hv_lines.iterrows():
        from_bus = pp.get_element_index(net, "bus", hv_line.from_bus)
        to_bus = pp.get_element_index(net, "bus", hv_line.to_bus)
        pp.create_line(net, from_bus, to_bus, length_km=hv_line.length,std_type=hv_line.std_type, name=hv_line.line_name, parallel=hv_line.parallel)

# show line table
net.line

,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,imax_ka,df,parallel,type,in_service
0,HV Line1,184-AL1/30-ST1A 110.0,18,32,30.0,0.1571,0.4,8.8,0.535,1.0,1,ol,True
1,HV Line2,184-AL1/30-ST1A 110.0,32,33,20.0,0.1571,0.4,8.8,0.535,1.0,1,ol,True
2,HV Line3,184-AL1/30-ST1A 110.0,33,35,30.0,0.1571,0.4,8.8,0.535,1.0,1,ol,True
3,HV Line4,184-AL1/30-ST1A 110.0,32,35,15.0,0.1571,0.4,8.8,0.535,1.0,1,ol,True
4,HV Line5,184-AL1/30-ST1A 110.0,34,35,25.0,0.1571,0.4,8.8,0.535,1.0,1,ol,True
5,HV Line6,184-AL1/30-ST1A 110.0,19,34,30.0,0.1571,0.4,8.8,0.535,1.0,2,ol,True


### Transformer

<img src="pics/example_network_trafos_hv.png" width="40%">

The 380/110 kV transformer connects the buses "Bus DB 1" and "Bus DB 2". We use the get_element_index function from the pandapower toolbox to find the bus indices of the buses with these names and create a transformer by directly specifying the parameters:

In [56]:
hv_bus = pp.get_element_index(net, "bus", "Bus DB 2")
lv_bus = pp.get_element_index(net, "bus", "Bus SB 1")
pp.create_transformer_from_parameters(net, hv_bus, lv_bus, sn_kva=300000, vn_hv_kv=380, vn_lv_kv=110, vscr_percent=0.06,
                                      vsc_percent=8, pfe_kw=0, i0_percent=0, tp_pos=0, shift_degree=0, name='EHV-HV-Trafo')

net.trafo # show trafo table

,name,std_type,hv_bus,lv_bus,sn_kva,vn_hv_kv,vn_lv_kv,vsc_percent,vscr_percent,pfe_kw,i0_percent,shift_degree,tp_side,tp_mid,tp_min,tp_max,tp_st_percent,tp_pos,in_service
0,EHV-HV-Trafo,None,13,17,300000.0,380.0,110.0,8.0,0.06,0.0,0.0,0.0,None,NaN,NaN,NaN,NaN,NaN,True


### Switches

<img src="pics/example_network_switches_hv.png" width="60%">

Now we create the switches to connect the buses in the transformer station. The switch configuration is stored in the following csv table:

In [57]:
hv_bus_sw = pd.read_csv('example_advanced/hv_bus_sw.csv', sep=';', header=0, decimal=',')
hv_bus_sw

,bus_name,from_bus,to_bus,type,et,closed
0,DB DS0,Double Busbar 2,Bus DB T0,DS,b,True
1,DB DS1,Double Busbar 1,Bus DB T1,DS,b,True
2,DB DS2,Double Busbar 2,Bus DB T3,DS,b,True
3,DB DS3,Double Busbar 1,Bus DB T3,DS,b,False
4,DB DS4,Double Busbar 2,Bus DB T5,DS,b,True
5,DB DS5,Double Busbar 1,Bus DB T5,DS,b,False
6,DB DS6,Double Busbar 2,Bus DB T7,DS,b,True
7,DB DS7,Double Busbar 1,Bus DB T7,DS,b,False
8,DB DS8,Double Busbar 2,Bus DB T9,DS,b,True
9,DB DS9,Double Busbar 1,Bus DB T9,DS,b,False


In [58]:
# Bus-bus switches
for _, switch in hv_bus_sw.iterrows():
    from_bus = pp.get_element_index(net, "bus", switch.from_bus)
    to_bus = pp.get_element_index(net, "bus", switch.to_bus)
    pp.create_switch(net, from_bus, to_bus, et=switch.et, closed=switch.closed, type=switch.type, name=switch.bus_name)

# Bus-line switches
hv_buses = net.bus[(net.bus.vn_kv == 380) | (net.bus.vn_kv == 110)].index
hv_ls = net.line[(net.line.from_bus.isin(hv_buses)) & (net.line.to_bus.isin(hv_buses))]
for _, line in hv_ls.iterrows():
        pp.create_switch(net, line.from_bus, line.name, et='l', closed=True, type='LBS', name='Switch %s - %s' % (net.bus.name.at[line.from_bus], line['name']))
        pp.create_switch(net, line.to_bus, line.name, et='l', closed=True, type='LBS', name='Switch %s - %s' % (net.bus.name.at[line.to_bus], line['name']))

# Trafo-line switches
pp.create_switch(net, pp.get_element_index(net, "bus", 'Bus DB 2'), pp.get_element_index(net, "trafo", 'EHV-HV-Trafo'), et='t', closed=True, type='LBS', name='Switch DB2 - EHV-HV-Trafo')
pp.create_switch(net, pp.get_element_index(net, "bus", 'Bus SB 1'), pp.get_element_index(net, "trafo", 'EHV-HV-Trafo'), et='t', closed=True, type='LBS', name='Switch SB1 - EHV-HV-Trafo')

# show switch table
net.switch

,bus,element,et,type,closed,name
0,1,2,b,DS,True,DB DS0
1,0,3,b,DS,True,DB DS1
2,1,5,b,DS,True,DB DS2
3,0,5,b,DS,False,DB DS3
4,1,7,b,DS,True,DB DS4
5,0,7,b,DS,False,DB DS5
6,1,9,b,DS,True,DB DS6
7,0,9,b,DS,False,DB DS7
8,1,11,b,DS,True,DB DS8
9,0,11,b,DS,False,DB DS9


### External Grid

<img src="pics/example_network_ext_grids_hv.png" width="40%">

We equip the high voltage side of the transformer with an external grid connection:

In [59]:
pp.create_ext_grid(net, pp.get_element_index(net, "bus", 'Double Busbar 1'), vm_pu=1.03, va_degree=0, name='External grid',
                   s_sc_max_mva=10000, rx_max=0.1, rx_min=0.1)

net.ext_grid # show external grid table

,name,bus,vm_pu,va_degree,in_service,s_sc_max_mva,rx_min,rx_max
0,External grid,0,1.03,0.0,True,10000.0,0.1,0.1


### Loads

<img src="pics/example_network_loads_hv.png" width="40%">

The five loads in the HV network are defined in the following csv file:

In [60]:
hv_loads = pd.read_csv('example_advanced/hv_loads.csv', sep=';', header=0, decimal=',')
hv_loads

,load_name,bus,p,q
0,MV Net 0,Bus SB 4,38000,6000
1,MV Net 1,Bus HV1,38000,6000
2,MV Net 2,Bus HV2,38000,6000
3,MV Net 3,Bus HV3,38000,6000
4,MV Net 4,Bus HV4,38000,6000


In [61]:
for _, load in hv_loads.iterrows():
    bus_idx = pp.get_element_index(net, "bus", load.bus)
    pp.create_load(net, bus_idx, p_kw=load.p, q_kvar=load.q, name=load.load_name)

# show load table
net.load

,name,bus,p_kw,q_kvar,sn_kva,scaling,in_service,type
0,MV Net 0,20,38000.0,6000.0,NaN,1.0,True,None
1,MV Net 1,32,38000.0,6000.0,NaN,1.0,True,None
2,MV Net 2,33,38000.0,6000.0,NaN,1.0,True,None
3,MV Net 3,34,38000.0,6000.0,NaN,1.0,True,None
4,MV Net 4,35,38000.0,6000.0,NaN,1.0,True,None


### Generator

<img src="pics/example_network_gens_hv.png" width="40%">

The voltage controlled generator is created with an active power of 100 MW (negative for generation) and a voltage set point of 1.03 per unit:

In [62]:
pp.create_gen(net, pp.get_element_index(net, "bus", 'Bus HV4'), vm_pu=1.03, p_kw=-100e3, name='Gas turbine')

# show generator table
net.gen

,name,bus,p_kw,vm_pu,sn_kva,min_q_kvar,max_q_kvar,scaling,in_service,type
0,Gas turbine,35,-100000.0,1.03,NaN,NaN,NaN,1.0,True,None


### Static generators

<img src="pics/example_network_sgens_hv.png" width="40%">

We create this wind park with an active power of 20 MW (negative for generation) and a reactive power of -4 Mvar. To classify the generation as a wind park, we set type to "WP":

In [63]:
pp.create_sgen(net, pp.get_element_index(net, "bus", 'Bus SB 5'), p_kw=-20e3, q_kvar=-4e3, sn_kva=45e3, 
               type='WP', name='Wind Park')

# show static generator table
net.sgen

,name,bus,p_kw,q_kvar,sn_kva,scaling,in_service,type,controllable
0,Wind Park,21,-20000.0,-4000.0,45000.0,1.0,True,WP,False


### Shunt

<img src="pics/example_network_shunts_hv.png" width="40%">

In [64]:
pp.create_shunt(net, pp.get_element_index(net, "bus", 'Bus HV1'), p_kw=0, q_kvar=-960, name='Shunt')

# show shunt table
net.shunt

,bus,name,q_kvar,p_kw,in_service
0,32,Shunt,-960.0,0.0,1


### External network equivalents


<img src="pics/example_network_ext_equi_hv.png" width="40%">

The two remaining elements are impedances and extended ward equivalents:

In [65]:
# Impedance
pp.create_impedance(net, pp.get_element_index(net, "bus", 'Bus HV3'), pp.get_element_index(net, "bus", 'Bus HV1'), 
                    r_pu=0.074873, x_pu=0.198872, sn_kva=100000, name='Impedance')

# show impedance table
net.impedance

,name,from_bus,to_bus,r_pu,x_pu,sn_kva,in_service
0,Impedance,34,32,0.074873,0.198872,100000.0,True


In [66]:
# xwards
pp.create_xward(net, pp.get_element_index(net, "bus", 'Bus HV3'), ps_kw=23942, qs_kvar=-12241.87, pz_kw=2814.571, 
                qz_kvar=0, r_ohm=0, x_ohm=12.18951, vm_pu=1.02616, name='XWard 1')
pp.create_xward(net, pp.get_element_index(net, "bus", 'Bus HV1'), ps_kw=3776, qs_kvar=-7769.979, pz_kw=9174.917, 
                qz_kvar=0, r_ohm=0, x_ohm=50.56217, vm_pu=1.024001, name='XWard 2')

# show xward table
net.xward

,name,bus,ps_kw,qs_kvar,qz_kvar,pz_kw,r_ohm,x_ohm,vm_pu,in_service
0,XWard 1,34,23942.0,-12241.870,0.0,2814.571,0.0,12.18951,1.026160,1.0
1,XWard 2,32,3776.0,-7769.979,0.0,9174.917,0.0,50.56217,1.024001,1.0


## Medium voltage level

### Buses

<img src="pics/example_network_buses_mv.png" width="50%">

In [67]:
pp.create_bus(net, name='Bus MV0 20kV', vn_kv=20, type='n')
for i in range(8):
    pp.create_bus(net, name='Bus MV%s' % i, vn_kv=10, type='n')

#show only medium voltage bus table
mv_buses = net.bus[(net.bus.vn_kv == 10) | (net.bus.vn_kv == 20)]
mv_buses

,name,vn_kv,type,zone,in_service
36,Bus MV0 20kV,20.0,n,None,True
37,Bus MV0,10.0,n,None,True
38,Bus MV1,10.0,n,None,True
39,Bus MV2,10.0,n,None,True
40,Bus MV3,10.0,n,None,True
41,Bus MV4,10.0,n,None,True
42,Bus MV5,10.0,n,None,True
43,Bus MV6,10.0,n,None,True
44,Bus MV7,10.0,n,None,True


### Lines

<img src="pics/example_network_lines_mv.png" width="50%">

In [68]:
mv_lines = pd.read_csv('example_advanced/mv_lines.csv', sep=';', header=0, decimal=',')
for _, mv_line in mv_lines.iterrows():
    from_bus = pp.get_element_index(net, "bus", mv_line.from_bus)
    to_bus = pp.get_element_index(net, "bus", mv_line.to_bus)
    pp.create_line(net, from_bus, to_bus, length_km=mv_line.length, std_type=mv_line.std_type, name=mv_line.line_name)

# show only medium voltage lines
net.line[net.line.from_bus.isin(mv_buses.index)]

,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,imax_ka,df,parallel,type,in_service
6,MV Line1,NA2XS2Y 1x185 RM/25 12/20 kV,37,38,1.5,0.161,0.117,273.0,0.362,1.0,1,cs,True
7,MV Line2,NA2XS2Y 1x185 RM/25 12/20 kV,38,39,1.5,0.161,0.117,273.0,0.362,1.0,1,cs,True
8,MV Line3,NA2XS2Y 1x185 RM/25 12/20 kV,39,40,1.5,0.161,0.117,273.0,0.362,1.0,1,cs,True
9,MV Line4,NA2XS2Y 1x185 RM/25 12/20 kV,40,41,1.5,0.161,0.117,273.0,0.362,1.0,1,cs,True
10,MV Line5,NA2XS2Y 1x185 RM/25 12/20 kV,41,42,1.5,0.161,0.117,273.0,0.362,1.0,1,cs,True
11,MV Line6,NA2XS2Y 1x185 RM/25 12/20 kV,42,43,1.5,0.161,0.117,273.0,0.362,1.0,1,cs,True
12,MV Line7,NA2XS2Y 1x185 RM/25 12/20 kV,43,44,1.5,0.161,0.117,273.0,0.362,1.0,1,cs,True
13,MV Line8,NA2XS2Y 1x185 RM/25 12/20 kV,37,44,1.5,0.161,0.117,273.0,0.362,1.0,1,cs,True


### 3 Winding Transformer

<img src="pics/example_network_trafos_mv.png" width="50%">

The three winding transformer transforms its high voltage level to two different lower voltage levels, in this case from 110 kV to 20 kV and 10 kV.

In [69]:
hv_bus = pp.get_element_index(net, "bus", "Bus HV2")
mv_bus = pp.get_element_index(net, "bus", "Bus MV0 20kV")
lv_bus = pp.get_element_index(net, "bus", "Bus MV0")
pp.create_transformer3w_from_parameters(net, hv_bus, mv_bus, lv_bus, vn_hv_kv=110, vn_mv_kv=20, vn_lv_kv=10, 
                                        sn_hv_kva=40000, sn_mv_kva=15000, sn_lv_kva=25000, vsc_hv_percent=10.1, 
                                        vsc_mv_percent=10.1, vsc_lv_percent=10.1, vscr_hv_percent=0.266667, 
                                        vscr_mv_percent=0.033333, vscr_lv_percent=0.04, pfe_kw=0, i0_percent=0, 
                                        shift_mv_degree=30, shift_lv_degree=30, tp_side="hv", tp_mid=0, tp_min=-8, 
                                        tp_max=8, tp_st_percent=1.25, tp_pos=0, name='HV-MV-MV-Trafo')

# show transformer3w table
net.trafo3w

,name,std_type,hv_bus,mv_bus,lv_bus,vn_hv_kv,vn_mv_kv,vn_lv_kv,sn_hv_kva,sn_mv_kva,...,i0_percent,shift_mv_degree,shift_lv_degree,tp_side,tp_mid,tp_min,tp_max,tp_st_percent,tp_pos,in_service
0,HV-MV-MV-Trafo,NaN,33,36,37,110.0,20.0,10.0,40000,15000,...,0.0,30.0,30.0,hv,0,-8,8,1.25,0,True


### Switches

In [70]:
# Bus-line switches
mv_buses = net.bus[(net.bus.vn_kv == 10) | (net.bus.vn_kv == 20)].index
mv_ls = net.line[(net.line.from_bus.isin(mv_buses)) & (net.line.to_bus.isin(mv_buses))]
for _, line in mv_ls.iterrows():
        pp.create_switch(net, line.from_bus, line.name, et='l', closed=True, type='LBS', name='Switch %s - %s' % (net.bus.name.at[line.from_bus], line['name']))
        pp.create_switch(net, line.to_bus, line.name, et='l', closed=True, type='LBS', name='Switch %s - %s' % (net.bus.name.at[line.to_bus], line['name']))

# open switch
open_switch_id = net.switch[(net.switch.name == 'Switch Bus MV5 - MV Line5')].index
net.switch.closed.loc[open_switch_id] = False

#show only medium voltage switch table
net.switch[net.switch.bus.isin(mv_buses)]

,bus,element,et,type,closed,name
48,37,6,l,LBS,True,Switch Bus MV0 - MV Line1
49,38,6,l,LBS,True,Switch Bus MV1 - MV Line1
50,38,7,l,LBS,True,Switch Bus MV1 - MV Line2
51,39,7,l,LBS,True,Switch Bus MV2 - MV Line2
52,39,8,l,LBS,True,Switch Bus MV2 - MV Line3
53,40,8,l,LBS,True,Switch Bus MV3 - MV Line3
54,40,9,l,LBS,True,Switch Bus MV3 - MV Line4
55,41,9,l,LBS,True,Switch Bus MV4 - MV Line4
56,41,10,l,LBS,True,Switch Bus MV4 - MV Line5
57,42,10,l,LBS,False,Switch Bus MV5 - MV Line5


### Loads

<img src="pics/example_network_loads_mv.png"  width="50%">

In [71]:
mv_loads = pd.read_csv('example_advanced/mv_loads.csv', sep=';', header=0, decimal=',')
for _, load in mv_loads.iterrows():
    bus_idx = pp.get_element_index(net, "bus", load.bus)
    pp.create_load(net, bus_idx, p_kw=load.p, q_kvar=load.q, name=load.load_name)

# show only medium voltage loads
net.load[net.load.bus.isin(mv_buses)]

,name,bus,p_kw,q_kvar,sn_kva,scaling,in_service,type
5,Further MV-Rings,37,6000.0,2000.0,NaN,1.0,True,None
6,Industry Load,36,18000.0,4000.0,NaN,1.0,True,None
7,LV Net 1,38,400.0,100.0,NaN,1.0,True,None
8,LV Net 2,39,400.0,60.0,NaN,1.0,True,None
9,LV Net 3,40,400.0,60.0,NaN,1.0,True,None
10,LV Net 5,42,400.0,60.0,NaN,1.0,True,None
11,LV Net 6,43,400.0,60.0,NaN,1.0,True,None
12,LV Net 7,44,400.0,60.0,NaN,1.0,True,None


### Static generators

<img src="pics/example_network_sgens_mv.png"  width="50%">

In [72]:
mv_sgens = pd.read_csv('example_advanced/mv_sgens.csv', sep=';', header=0, decimal=',')
for _, sgen in mv_sgens.iterrows():
    bus_idx = pp.get_element_index(net, "bus", sgen.bus)
    pp.create_sgen(net, bus_idx, p_kw=sgen.p, q_kvar=sgen.q, sn_kva=sgen.sn, type=sgen.type, name=sgen.sgen_name)

# show only medium voltage static generators
net.sgen[net.sgen.bus.isin(mv_buses)]

,name,bus,p_kw,q_kvar,sn_kva,scaling,in_service,type,controllable
1,Biogas plant,43,-500.0,0.0,750.0,1.0,True,SGEN,False
2,Further MV Generator,37,-500.0,-50.0,1000.0,1.0,True,SGEN,False
3,Industry Generator,36,-15000.0,-3000.0,20000.0,1.0,True,SGEN,False
4,PV Park,42,-2000.0,-100.0,5000.0,1.0,True,PV,False


## Low voltage level

### Busses

<img src="pics/example_network_buses_lv.png" width="50%">

In [73]:
pp.create_bus(net, name='Bus LV0', vn_kv=0.4, type='n')
for i in range(1, 6):
    pp.create_bus(net, name='Bus LV1.%s' % i, vn_kv=0.4, type='m')
for i in range(1, 5):
    pp.create_bus(net, name='Bus LV2.%s' % i, vn_kv=0.4, type='m')
pp.create_bus(net, name='Bus LV2.2.1', vn_kv=0.4, type='m')
pp.create_bus(net, name='Bus LV2.2.2', vn_kv=0.4, type='m')

# show only low voltage buses
lv_buses = net.bus[net.bus.vn_kv == 0.4]
lv_buses

,name,vn_kv,type,zone,in_service
45,Bus LV0,0.4,n,None,True
46,Bus LV1.1,0.4,m,None,True
47,Bus LV1.2,0.4,m,None,True
48,Bus LV1.3,0.4,m,None,True
49,Bus LV1.4,0.4,m,None,True
50,Bus LV1.5,0.4,m,None,True
51,Bus LV2.1,0.4,m,None,True
52,Bus LV2.2,0.4,m,None,True
53,Bus LV2.3,0.4,m,None,True
54,Bus LV2.4,0.4,m,None,True


### Lines

<img src="pics/example_network_lines_lv.png" width="50%">

In [78]:
# create lines
lv_lines = pd.read_csv('example_advanced/lv_lines.csv', sep=';', header=0, decimal=',')
for _, lv_line in lv_lines.iterrows():
    from_bus = pp.get_element_index(net, "bus", lv_line.from_bus)
    to_bus = pp.get_element_index(net, "bus", lv_line.to_bus)
    pp.create_line(net, from_bus, to_bus, length_km=lv_line.length, std_type=lv_line.std_type, name=lv_line.line_name)

# show only low voltage lines
net.line[net.line.from_bus.isin(lv_buses.index)]

,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,imax_ka,df,parallel,type,in_service
14,LV Line1.1,NAYY 4x120 SE,45,46,0.08,0.2250,0.08,264.0,0.242,1.0,1,cs,True
15,LV Line1.2,NAYY 4x120 SE,46,47,0.08,0.2250,0.08,264.0,0.242,1.0,1,cs,True
16,LV Line1.3,NAYY 4x120 SE,47,48,0.08,0.2250,0.08,264.0,0.242,1.0,1,cs,True
17,LV Line1.4,NAYY 4x120 SE,48,49,0.08,0.2250,0.08,264.0,0.242,1.0,1,cs,True
18,LV Line1.6,NAYY 4x120 SE,49,50,0.08,0.2250,0.08,264.0,0.242,1.0,1,cs,True
19,LV Line2.1,NAYY 4x120 SE,45,51,0.12,0.2250,0.08,264.0,0.242,1.0,1,cs,True
20,LV Line2.2,NAYY 4x120 SE,51,52,0.12,0.2250,0.08,264.0,0.242,1.0,1,cs,True
21,LV Line2.3,15-AL1/3-ST1A 0.4,52,53,0.12,1.8769,0.35,11.0,0.105,1.0,1,ol,True
22,LV Line2.4,15-AL1/3-ST1A 0.4,53,54,0.12,1.8769,0.35,11.0,0.105,1.0,1,ol,True
23,LV Line2.2.1,15-AL1/3-ST1A 0.4,52,55,0.12,1.8769,0.35,11.0,0.105,1.0,1,ol,True


### Transformer

<img src="pics/example_network_trafos_lv.png" width="50%">

In [79]:
hv_bus = pp.get_element_index(net, "bus", "Bus MV4")
lv_bus = pp.get_element_index(net, "bus","Bus LV0")
pp.create_transformer_from_parameters(net, hv_bus, lv_bus, sn_kva=400, vn_hv_kv=10, vn_lv_kv=0.4, vscr_percent=1.325, vsc_percent=4, pfe_kw=0.95, i0_percent=0.2375, tp_side="hv", tp_mid=0, tp_min=-2, tp_max=2, tp_st_percent=2.5, tp_pos=0, shift_degree=150, name='MV-LV-Trafo')

#show only low voltage transformer
net.trafo[net.trafo.lv_bus.isin(lv_buses.index)]

,name,std_type,hv_bus,lv_bus,sn_kva,vn_hv_kv,vn_lv_kv,vsc_percent,vscr_percent,pfe_kw,i0_percent,shift_degree,tp_side,tp_mid,tp_min,tp_max,tp_st_percent,tp_pos,in_service
1,MV-LV-Trafo,None,41,45,400.0,10.0,0.4,4.0,1.325,0.95,0.2375,150.0,hv,0.0,-2.0,2.0,2.5,0.0,True
2,MV-LV-Trafo,None,41,45,400.0,10.0,0.4,4.0,1.325,0.95,0.2375,150.0,hv,0.0,-2.0,2.0,2.5,0.0,True


### Switches

In [80]:
# Bus-line switches
lv_ls = net.line[(net.line.from_bus.isin(lv_buses)) & (net.line.to_bus.isin(lv_buses))]
for _, line in lv_ls.iterrows():
        pp.create_switch(net, line.from_bus, line.name, et='l', closed=True, type='LBS', name='Switch %s - %s' % (net.bus.name.at[line.from_bus], line['name']))
        pp.create_switch(net, line.to_bus, line.name, et='l', closed=True, type='LBS', name='Switch %s - %s' % (net.bus.name.at[line.to_bus], line['name']))

# Trafo-line switches
pp.create_switch(net, pp.get_element_index(net, "bus", 'Bus MV4'), pp.get_element_index(net, "trafo", 'MV-LV-Trafo'), et='t', closed=True, type='LBS', name='Switch MV4 - MV-LV-Trafo')
pp.create_switch(net, pp.get_element_index(net, "bus", 'Bus LV0'), pp.get_element_index(net, "trafo", 'MV-LV-Trafo'), et='t', closed=True, type='LBS', name='Switch LV0 - MV-LV-Trafo')

# show only low vvoltage switches
net.switch[net.switch.bus.isin(lv_buses.index)]

UserWarning: Duplicate trafo names for MV-LV-Trafo

### Loads

<img src="pics/example_network_loads_lv.png" width="50%">

In [81]:
lv_loads = pd.read_csv('example_advanced/lv_loads.csv', sep=';', header=0, decimal=',')
for _, load in lv_loads.iterrows():
    bus_idx = pp.get_element_index(net, "bus", load.bus)
    pp.create_load(net, bus_idx, p_kw=load.p, q_kvar=load.q, name=load.load_name)
    
# show only low voltage loads
net.load[net.load.bus.isin(lv_buses.index)]

,name,bus,p_kw,q_kvar,sn_kva,scaling,in_service,type
13,Further LV-Feeders Load,45,100.0,10.0,NaN,1.0,True,None
14,Residential Load,46,10.0,3.0,NaN,1.0,True,None
15,Residential Load(1),47,10.0,3.0,NaN,1.0,True,None
16,Residential Load(2),48,10.0,3.0,NaN,1.0,True,None
17,Residential Load(3),49,10.0,3.0,NaN,1.0,True,None
18,Residential Load(4),50,10.0,3.0,NaN,1.0,True,None
19,Rural Load,51,10.0,3.0,NaN,1.0,True,None
20,Rural Load(1),52,10.0,3.0,NaN,1.0,True,None
21,Rural Load(2),53,10.0,3.0,NaN,1.0,True,None
22,Rural Load(3),54,10.0,3.0,NaN,1.0,True,None


### Static generators

<img src="pics/example_network_sgens_lv.png"  width="50%">

In [82]:
lv_sgens = pd.read_csv('example_advanced/lv_sgens.csv', sep=';', header=0, decimal=',')
for _, sgen in lv_sgens.iterrows():
    bus_idx = pp.get_element_index(net, "bus", sgen.bus)
    pp.create_sgen(net, bus_idx, p_kw=sgen.p, q_kvar=sgen.q, sn_kva=sgen.sn, type=sgen.type, name=sgen.sgen_name)

# show only low voltage static generators
net.sgen[net.sgen.bus.isin(lv_buses.index)]

,name,bus,p_kw,q_kvar,sn_kva,scaling,in_service,type,controllable
5,PV,46,-6.0,0.0,12.0,1.0,True,PV,False
6,PV(1),48,-5.0,0.0,10.0,1.0,True,PV,False
7,PV(2),53,-5.0,0.0,10.0,1.0,True,PV,False
8,PV(3),54,-5.0,0.0,10.0,1.0,True,PV,False
9,PV(4),55,-5.0,0.0,10.0,1.0,True,PV,False
10,PV(5),56,-5.0,0.0,10.0,1.0,True,PV,False


## Run a Power Flow

In [92]:
pp.runpp(net, calculate_voltage_angles=True, init="dc")
net

This pandapower network includes the following parameter tables:
   - line (36 elements)
   - sgen (11 elements)
   - ext_grid (1 elements)
   - bus (57 elements)
   - impedance (1 elements)
   - trafo (3 elements)
   - switch (66 elements)
   - xward (2 elements)
   - trafo3w (1 elements)
   - gen (1 elements)
   - load (37 elements)
   - shunt (1 elements)
 and the following results tables:
   - res_xward (2 elements)
   - res_trafo3w (1 elements)
   - res_load (37 elements)
   - res_gen (1 elements)
   - res_impedance (1 elements)
   - res_sgen (11 elements)
   - res_shunt (1 elements)
   - res_bus (57 elements)
   - res_trafo (3 elements)
   - res_line (36 elements)
   - res_ext_grid (1 elements)